In [ ]:
from flask import Flask, request, jsonify, make_response
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
import jwt
import datetime
from functools import wraps
import uuid

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///web.db'
bcrypt = Bcrypt(app)

db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    public_id = db.Column(db.String(50), unique=True)
    name = db.Column(db.String(50))
    password = db.Column(db.String(80))
    admin = db.Column(db.Boolean)

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.String(50))
    public = db.Column(db.Boolean)
    user_id = db.Column(db.Integer)

    

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'])
            current_user = User.query.filter_by(public_id=data['public_id']).first()
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated


########################
#USERS

#create a user
@app.route('/user', methods=['POST'])
def create_user():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(public_id=str(uuid.uuid4()), name=data['name'], password=hashed_password, admin=False)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'Successfully created!'})

#get all users
@app.route('/user', methods=['GET'])
@token_required
def get_all_users(current_user):

    if not current_user.admin:
        return jsonify({'message' : 'Sorry! Cannot perform the function!'})

    users = User.query.all()

    output = []

    for user in users:
        user_data = {}
        user_data['public_id'] = str(user.public_id)
        user_data['name'] = user.name
        user_data['password'] = str(user.password)
        user_data['admin'] = user.admin
        output.append(user_data)

    return jsonify({'users' : output})

#get one user
@app.route('/user/<public_id>', methods=['GET'])
@token_required
def get_one_user(current_user, public_id):
    
    if not current_user.admin:
        return jsonify({'message' : 'Sorry! Cannot perform the function!'})

    user = User.query.filter_by(public_id=public_id).first()

    if not user:
        return jsonify({'message' : 'Sorry! No user found!'})

    user_data = {}
    user_data['public_id'] = str(user.public_id)
    user_data['name'] = user.name
    user_data['password'] = str(user.password)
    user_data['admin'] = user.admin

    return jsonify({'user' : user_data})

#promote a user
@app.route('/user/<public_id>', methods=['PUT'])
@token_required
def promote_user(current_user, public_id):
    
    if not current_user.admin:
        return jsonify({'message' : 'Sorry! Cannot perform the function!'})

    user = User.query.filter_by(public_id=public_id).first()

    if not user:
        return jsonify({'message' : 'Sorry! No user found!'})

    user.admin = True
    db.session.commit()

    return jsonify({'message' : 'Successfully promoted!'})

#delete a user
@app.route('/user/<public_id>', methods=['DELETE'])
@token_required
def delete_user(current_user, public_id):
    
    if not current_user.admin:
        return jsonify({'message' : 'Sorry! Cannot perform the function!'})

    user = User.query.filter_by(public_id=public_id).first()

    if not user:
        return jsonify({'message' : 'Sorry! No user found!'})

    db.session.delete(user)
    db.session.commit()

    return jsonify({'message' : 'Successfully deleted!'})

#login - for token
@app.route('/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(name=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password):
        token = jwt.encode({'public_id' : user.public_id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])

        return jsonify({'token' : token.decode('UTF-8')})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

########################
#POSTS

#create a post
@app.route('/post', methods=['POST'])
@token_required
def create_post(current_user):
    data = request.get_json()

    new_post = Post(text=data['text'], public=False, user_id=current_user.id)
    db.session.add(new_post)
    db.session.commit()

    return jsonify({'message' : "Successfully created!"})

#get all posts
@app.route('/post', methods=['GET'])
@token_required
def get_all_posts(current_user):
    posts = Post.query.filter_by(user_id=current_user.id).all()

    output = []

    for post in posts:
        post_data = {}
        post_data['id'] = post.id
        post_data['text'] = post.text
        post_data['public'] = post.public
        output.append(post_data)

    return jsonify({'posts' : output})

#get one post
@app.route('/post/<post_id>', methods=['GET'])
@token_required
def get_one_post(current_user, post_id):
    post = Post.query.filter_by(id=post_id, user_id=current_user.id).first()

    if not post:
        return jsonify({'message' : 'Sorry! No post found!'})

    post_data = {}
    post_data['id'] = post.id
    post_data['text'] = post.text
    post_data['public'] = post.public

    return jsonify(post_data)

#publish a post
@app.route('/post/<post_id>', methods=['PUT'])
@token_required
def publish_post(current_user, post_id):
    post = Post.query.filter_by(id=post_id, user_id=current_user.id).first()

    if not post:
        return jsonify({'message' : 'Sorry! No post found!'})

    post.public = True
    db.session.commit()

    return jsonify({'message' : 'Successfully Published!'})

#delete a post
@app.route('/post/<post_id>', methods=['DELETE'])
@token_required
def delete_post(current_user, post_id):
    post = Post.query.filter_by(id=post_id, user_id=current_user.id).first()

    if not post:
        return jsonify({'message' : 'Sorry! No post found!'})

    db.session.delete(post)
    db.session.commit()

    return jsonify({'message' : 'Successfully deleted!'})


########################

if __name__ == '__main__':
    app.run('localhost', port=2416)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/sammy/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:2416/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2022 16:07:59] "DELETE /post/3 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 16:08:10] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 16:08:13] "GET /post HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 16:08:17] "DELETE /post/3 HTTP/1.1" 200 -
